# Simple retrieval augmented generation with OpenAI

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [1]:
APPLY = True
SAMPLE_COLLECTION_NAME = 'sample_simple_rag'
COLLECTION_NAME = '<var:table_name>' if not APPLY else 'docs'
ID_FIELD = '<var:id_field>' if not APPLY else 'id'
OUTPUT_PREFIX = 'outputs__'
EAGER = False

In [5]:
from superduper import superduper, CFG
import os

db = superduper()

2025-Feb-23 13:55:42.33| INFO     | Duncans-MBP.fritz.box| superduper.misc.importing:13   | Loading plugin: mongodb
2025-Feb-23 13:55:42.33| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:50   | Building Data Layer
2025-Feb-23 13:55:42.33| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:67   | Data Layer built
2025-Feb-23 13:55:42.34| INFO     | Duncans-MBP.fritz.box| superduper.backends.base.cluster:113  | Cluster initialized in 0.01 seconds.
2025-Feb-23 13:55:42.34| INFO     | Duncans-MBP.fritz.box| superduper.base.build:146  | Configuration: 
 +----------------+-----------------------------------+
| Configuration  |               Value               |
+----------------+-----------------------------------+
|  Data Backend  | mongodb://localhost:27017/test_db |
| Artifact Store |   filesystem://./artifact_store   |
+----------------+-----------------------------------+


In [6]:
db.drop(True, True)
db = superduper()

2025-Feb-23 13:55:43.87| INFO     | Duncans-MBP.fritz.box| superduper.misc.importing:13   | Loading plugin: mongodb
2025-Feb-23 13:55:43.87| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:50   | Building Data Layer
2025-Feb-23 13:55:43.87| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:67   | Data Layer built
2025-Feb-23 13:55:43.89| INFO     | Duncans-MBP.fritz.box| superduper.backends.base.cluster:113  | Cluster initialized in 0.01 seconds.
2025-Feb-23 13:55:43.89| INFO     | Duncans-MBP.fritz.box| superduper.base.build:146  | Configuration: 
 +----------------+-----------------------------------+
| Configuration  |               Value               |
+----------------+-----------------------------------+
|  Data Backend  | mongodb://localhost:27017/test_db |
| Artifact Store |   filesystem://./artifact_store   |
+----------------+-----------------------------------+


In [7]:
import json
import requests
import io
from superduper import logging


def getter():
    logging.info('Downloading data...')
    response = requests.get('https://superduperdb-public-demo.s3.amazonaws.com/text.json')
    logging.info('Downloading data... (Done)')
    data = json.loads(response.content.decode('utf-8'))
    return [{'x': r} for r in data]

In [8]:
if APPLY:
    data = getter()

2025-Feb-23 13:55:48.16| INFO     | Duncans-MBP.fritz.box| __main__:8    | Downloading data...
2025-Feb-23 13:55:49.04| INFO     | Duncans-MBP.fritz.box| __main__:10   | Downloading data... (Done)


<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [9]:
if APPLY:
    from superduper import Document, Table
    table = Table(COLLECTION_NAME, fields={'x': 'str'})
    db.apply(table, force=True)
    ids = db.execute(db[COLLECTION_NAME].insert(data))

2025-Feb-23 13:55:50.43| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-23 13:55:50.43| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Table')) from metadata...
2025-Feb-23 13:55:50.43| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Table:abc to cache
2025-Feb-23 13:55:50.48| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, docs) not found in cache, loading from db
2025-Feb-23 13:55:50.48| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'docs')) from metadata...
2025-Feb-23 13:55:50.48| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:369  | Found new Table:docs:e00fdad004614226
2025-Feb-23 13:55:50.48| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:78   | Found these changes and/ or additions that need to be made:
2025-Feb-23 13:55:50.48| INF

Create plugin:

In [10]:
from superduper import Plugin

plugin = Plugin('rag-plugin', path='./rag_plugin.py')

<!-- TABS -->
## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [11]:
from superduper import Listener
from rag_plugin import Chunker

upstream_listener = Listener(
    model=Chunker(identifier='chunker'),
    select=db[COLLECTION_NAME],
    key='x',
    identifier='chunker',
    flatten=True,
    upstream=[plugin],
)

In [12]:
if APPLY and EAGER:
    db.apply(upstream_listener, force=True)

## Select outputs of upstream listener

:::note
This is useful if you have performed a first step, such as pre-computing 
features, or chunking your data. You can use this query to 
operate on those outputs.
:::

<!-- TABS -->
## Build text embedding model

OpenAI:

In [17]:
import os

from superduper_openai import OpenAIEmbedding

openai_embedding = OpenAIEmbedding(
    identifier='text-embedding',
    model='text-embedding-ada-002',
    datatype='vector[float:1536]',
)

## Create vector-index

In [18]:
from superduper import VectorIndex, Listener

vector_index_name = 'vectorindex'

vector_index = VectorIndex(
    vector_index_name,
    indexing_listener=Listener(
        key=upstream_listener.outputs,
        select=db[upstream_listener.outputs],
        model=openai_embedding,
        identifier='embeddinglistener',
        upstream=[upstream_listener],
    )
)

In [19]:
if APPLY and EAGER:
    db.apply(vector_index, force=True)

<!-- TABS -->
## Build LLM

In [21]:
from superduper_openai import OpenAIChatCompletion


llm_openai = OpenAIChatCompletion(
    identifier='llm-model',
    model='gpt-3.5-turbo',
    datatype='str',
)

## Answer question with LLM

In [22]:
from superduper import model
from rag_plugin import RAGModel


prompt_template = (
    "Use the following context snippets, these snippets are not ordered!, Answer the question based on this context.\n"
    "These snippets are samples from our internal data-repositories, and should be used exclusively and as a matter"
    " of priority to answer the question. Please answer in 20 words or less.\n\n"
    "{context}\n\n"
    "Here's the question: {query}"
)

rag = RAGModel(
    'simple_rag',
    select=db[upstream_listener.outputs].select().like({upstream_listener.outputs: '<var:query>'}, vector_index=vector_index_name, n=5),
    prompt_template=prompt_template,
    key=upstream_listener.outputs,
    llm=llm_openai,
)

In [23]:
if APPLY and EAGER:
    db.apply(rag, force=True)

By applying the RAG model to the database, it will subsequently be accessible for use in other services.

In [24]:
from superduper import Streamlit, Plugin
from rag_plugin import demo_func

demo = Streamlit('simple-rag-demo', demo_func=demo_func, upstream=[plugin])

In [25]:
from superduper import Application

app = Application(
    'simple-rag-app',
    components=[
        upstream_listener,
        vector_index,
        rag,
        demo,
    ]
)

In [26]:
if APPLY:
    db.apply(app, force=True)

2025-Feb-23 13:56:33.19| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-23 13:56:33.19| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Table')) from metadata...
2025-Feb-23 13:56:33.19| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Table:abc to cache
2025-Feb-23 13:56:33.20| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-23 13:56:33.20| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Table')) from metadata...
2025-Feb-23 13:56:33.21| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Table:abc to cache
2025-Feb-23 13:56:33.22| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Table) not found in cache, loading from db
2025-Feb-23 13:56:33.22

[2025-02-23 13:56:36] httpx INFO HTTP Request: GET https://api.openai.com/v1/models "HTTP/1.1 200 OK"
[2025-02-23 13:56:37] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"                                                                                                                     | 0/4 [00:00<?, ?it/s]
[2025-02-23 13:56:39] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"                                                                                                             | 1/4 [00:02<00:07,  2.35s/it]
[2025-02-23 13:56:41] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"████                                                                                                         | 2/4 [00:04<00:04,  2.06s/it]
[2025-02-23 13:56:42] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"██████████████████████████████████████████████████████

2025-Feb-23 13:56:43.29| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, _outputs__embeddinglistener__53e701637b7346f5) not found in cache, loading from db
2025-Feb-23 13:56:43.29| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', '_outputs__embeddinglistener__53e701637b7346f5')) from metadata...
2025-Feb-23 13:56:43.30| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:_outputs__embeddinglistener__53e701637b7346f5:77ad2bbcdb6d4f13 to cache
2025-Feb-23 13:56:44.06| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:387  | Component (Table, Job) not found in cache, loading from db
2025-Feb-23 13:56:44.06| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:393  | Load (('Table', 'Job')) from metadata...
2025-Feb-23 13:56:44.06| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:405  | Adding Table:Job:jkl to cache
2025-Feb-23 13:56:44.12| INFO     | Duncans-MBP.f

In [27]:
if APPLY:
    rag = db.load('RAGModel', 'simple_rag')
    print(rag.predict('Tell me about the project'))

2025-Feb-23 13:56:51.36| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:608  | Getting vector-index
2025-Feb-23 13:56:51.36| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:616  | {}


[2025-02-23 13:56:51] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
[2025-02-23 13:56:53] httpx INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Superduper is a project for building, deploying, and managing AI models with a data layer, compute layer, and component tracking.


You can now load the model elsewhere and make predictions using the following command.

## Create template

In [28]:
from superduper import Template, Table, Schema
from superduper.components.dataset import RemoteData

template = Template(
    'simple_rag',
    template=app,
    substitutions={
        COLLECTION_NAME: 'table_name',
        'text-embedding-ada-002': 'embedding_model',
        'gpt-3.5-turbo': 'llm_model',
    },
    template_variables=['table_name', 'id_field', 'embedding_model', 'llm_model'],
    default_tables=[
        Table(
            'sample_simple_rag',
            fields={'x': 'str'},
            data=RemoteData(
                'superduper-docs',
                getter=getter,
            )
        ),
    ],
    types={
        'id_field': {
            'type': 'str',
            'default': '_id',
        },
        'embedding_model': {
            'type': 'str',
            'default': 'text-embedding-ada-002',
            'choices': ['text-embedding-ada-002', 'nomic-embed-text:latest'],
        },
        'llm_model': {
            'type': 'str',
            'default': 'gpt-3.5-turbo',
            'choices': ['gpt-3.5-turbo', 'gpt-4-turbo', 'llama3.1:8b']
        },
        'table_name': {
            'type': 'str',
            'default': SAMPLE_COLLECTION_NAME,
        }
    },
    schema={
        "id_field": "id_field",
        "embedding_model": "embedding_model",
        "llm_model": "llm_model",
        "table_name": "table_name",
    },
    db=db
)

In [ ]:
template.export('.')